In [1]:
%load_ext watermark
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from myst_nb import glue
from slugify import slugify

from plastockconf import name_zones, name_frequentation, name_situation
from plastockconf import name_substrate, name_distance, table_css_styles

from plastock import add_table_to_page, capitalize_x_tick_labels, capitalize_x_and_y_axis_labels, capitalize_legend_components, attribute_summary

import reportclass as rc
import setvariables as conf_

def translate_describe(x, value_column):
    described = x.to_dict()
    described.pop("count")
    described["moyenne"] = described.pop("mean")
    described["écart-type"] = described.pop("std")
    df = pd.DataFrame(described.items())
    df.set_index(0, inplace=True)
    df.rename(columns={1:value_column}, inplace=True)
    df.index.name = None
    
    
    return df
format_kwargs = dict(precision=2, thousands="'", decimal=",")

def select_a_set_of_values(data, column_one, threshold_data):
    
    return data.loc[data[column_one] > threshold_data]

glue('blank_caption', " ", display=False)

# Déchets Sauvages

In [2]:
new_data = pd.read_csv("data/macro_current.csv")
beach_data = pd.read_csv("data/pstock_beaches_current.csv")
codes = pd.read_csv('data/end_pipe/codes.csv').set_index('code')

new_column_names = {
    "Position":"position",
    "Substrat":"substrat",
    "Date":"date",
    "Code":"code",
    "Quantité":"quantité",
    "Aire":"area"
}

length_key = beach_data[["Plage","length"]].drop_duplicates("Plage").set_index("Plage")
work_data = new_data[["Plage", *new_column_names.keys()]].copy()
work_data.rename(columns=new_column_names, inplace=True)
work_data["length"] = work_data.Plage.apply(lambda x: length_key.loc[x, "length"])
work_data["slug"] = work_data.Plage.apply(lambda x: slugify(x))
work_data["echantillon"] = list(zip(work_data.slug, work_data['date']))
work_data['date'] = pd.to_datetime(work_data["date"], format="mixed", dayfirst=True)
work_data.dropna(inplace=True)
work_data[["position", "substrat"]] = work_data[["position", "substrat"]].astype("int")
work_data['échantillon'] = work_data['echantillon']
work_data.drop(['echantillon'], inplace=True, axis=1)
work_data = work_data.groupby(['échantillon', 'Plage', 'position', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})
work_data['pcs/m'] = work_data['quantité']/work_data['length']

In [3]:
new_names = {x: " " for x in work_data.columns}
new_names = {
    'échantillon': 'loc_date',
    'pcs/m': 'pcs_m'
}

f = pd.read_csv('data/u_pstk.csv')

lone = work_data.slug.unique()
ltwo = f.slug.unique()

change_names = ['preverenges', 'tolochenaz', 'versoix', 'vidy', 'cully']

changeus = work_data[work_data.slug.isin(change_names)].copy()
donotchange = work_data[~work_data.slug.isin(change_names)].copy()

new_slug = {
    'cully': 'cully-p',
    'preverenges': 'preverenges-p',
    'tolochenaz': 'tolochenaz-p',
    'versoix':'versoix-p',
    'vidy': 'vidy-p'}

# they have the same name as locations in iqaasl
changeus['new_slug'] = changeus.slug.apply(lambda x: new_slug[x])
changeus['slug'] = changeus.new_slug
changeus.drop('new_slug', inplace=True, axis=1)
wd_nn = pd.concat([changeus, donotchange])

# codes in the plastock data
pcodes = wd_nn.code.unique()

# identify and remove codes for which there is no defintion
t = [x for x in pcodes if x not in codes.index]
wd_ni = wd_nn[~wd_nn.code.isin(t)].copy()

# aggregate to grfags, gfoams, gcaps
ti = rc.use_gfrags_gfoams_gcaps(wd_ni, codes)
ti = ti.groupby(['échantillon', 'Plage', 'position', 'substrat', 'date', 'length', 'slug', 'code'], as_index=False).agg({'quantité':'sum'})

# prvious source of plastock data with city labels


# make city map and format plastock data to work with report class
city_map = f[["slug", "city"]].drop_duplicates()
city_map.loc[city_map.slug == 'savoniere', 'slug'] = 'savonniere'
city_map.set_index('slug', inplace=True)

# adding and renaming columns according to reportclass requirements
ti['city'] = ti.slug.apply(lambda x: city_map.loc[x])
ti['feature_name'] = 'lac-leman'
ti['feature_type'] = 'l'
ti['parent_boundary'] = 'rhone'
ti.rename(columns={**new_names,'quantité': 'quantity'}, inplace=True)
ti['pcs_m'] = ti.quantity/ti.length


twork_cols = ['loc_date', 'date', 'slug',
       'code', 'quantity', 'city', 'feature_name', 'feature_type',
       'parent_boundary', 'pcs_m']

ti_work = ti[twork_cols].copy()

# previous results
ghi = pd.read_csv('data/end_pipe/iqaasl.csv')

ti_work['project']='Plastock'

txi = pd.concat([ghi, ti_work])
txi.reset_index(inplace=True)

language_maps = rc.language_maps()
boundaries = dict(start_date="2021-12-31", end_date="2023-01-01", feature_name="lac-leman", language="fr")
top_label= ['feature_name', 'lac-leman']

tiii = rc.ReportClass(ti_work.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps, top_label=top_label)
header = tiii.a_short_description.fillna('.')
comp_summary = tiii.summarize_feature_labels(feature='feature_name')
most_common, weight = tiii.most_common

## Situation

In [4]:
rc.translated_and_style_for_display(header.copy(),  tiii.lang_maps[tiii.language], tiii.language, gradient=False)

,Report
Type,feature_name
Name,lac-leman
Start,2021-12-31
End,2023-01-01
Échantillons,98
Quantité,27'493


In [5]:
rc.translated_and_style_for_display(comp_summary.copy(),  tiii.lang_maps[tiii.language], tiii.language, gradient=False)

,Pcs_m
label,Lac-leman
25%,"0,78"
50%,"2,00"
75%,"4,57"
Échantillons,98
Max,"18,26"
Moyenne,"3,42"
Min,"0,10"
Écart-type,"3,66"
Total,27'493


## Les plus courant

In [6]:
rc.translated_and_style_for_display(most_common.copy(),  tiii.lang_maps[tiii.language], tiii.language, gradient=False)

,Quantité,% du total,Pcs/m,Taux d'échec
"Fragments de plastique: g80, g79, g78, g75",11'221,"0,41","0,32","0,97"
Mégots et filtres à cigarettes,3'089,"0,11","0,12","0,79"
"Emballages de bonbons, de snacks",2'080,"0,08","0,13","0,74"
Fragments de plastique angulaires <5mm,1'926,"0,07","0,17","0,41"
Pellets industriels (gpi),1'526,"0,06","0,10","0,36"
"Fragments de polystyrène expansé: g76, g81, g82, g83",1'399,"0,05","0,09","0,72"
"Couvercles en plastique bouteille: g21, g22, g23, g24",1'070,"0,04","0,06","0,65"
Coton-tige,1'040,"0,04","0,11","0,54"
Mousse de plastique pour l'isolation thermique,406,"0,01","0,05","0,38"
Déchets de construction en plastique,380,"0,01","0,04","0,24"


### Les plus courant par ville

In [7]:
w_df = tiii.w_df.copy()
cities = w_df.city.unique()
cone = cities[:12]
ctwo = cities[12:]

t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common.index))], feature_name='city', object_column='code', table_split=cone)
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Amphion,Anthy,Aubonne,Bouveret,Clarens,Crans,Cully,Excenevex,Genève,Gland,Grangettes,Hermance,Cumulé
Fragments de plastique angulaires <5mm,"0,19",nan,"0,03","0,68","0,02",nan,"0,06","1,61","0,46",nan,"0,69",nan,"0,27"
Pellets industriels (gpi),"0,01","0,01",nan,"0,42","0,05",nan,nan,"1,20","0,27","0,22","0,27",nan,"0,17"
Mégots et filtres à cigarettes,"0,13","0,08","0,19","0,43","1,19",nan,"0,36","0,36","0,36","0,13","0,34","0,17","0,27"
"Emballages de bonbons, de snacks","0,43","0,01","0,08","0,43","0,24","0,07",nan,"0,14","0,16",nan,"1,94","0,06","0,22"
Bâtonnets de sucette,"0,12","0,02","0,02","0,15","0,08","0,05",nan,"0,06",nan,nan,"0,05","0,09","0,06"
Mousse de plastique pour l'isolation thermique,"0,05",nan,"0,06","0,02","0,31",nan,nan,"0,03","0,02","0,26","0,68","0,03","0,07"
Déchets de construction en plastique,"0,04","0,02",nan,"0,49","0,05",nan,nan,nan,"0,03","0,04","0,33",nan,"0,04"
Coton-tige,"0,70","0,04",nan,"0,24","0,27",nan,nan,"0,33","0,03","0,09","0,55",nan,"0,19"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,65",nan,"0,07","0,41","0,05",nan,"0,14","0,06","0,06","0,04","0,33","0,02","0,10"
"Fragments de polystyrène expansé: g76, g81, g82, g83","0,45","0,04","0,03","0,30","0,21",nan,nan,"0,09","0,05","0,48","1,31","0,02","0,14"


In [8]:
t = rc.a_cumulative_report(w_df[(w_df.code.isin(most_common.index))], feature_name='city', object_column='code', table_split=ctwo)
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Lugrin,Lutry,Meillerie,Préverenges,Rolle,Saint-disdille,Savonière,Tolochenaz,Tougues,Versoix,Vevey,Vidy,Cumulé
Fragments de plastique angulaires <5mm,"0,01","0,88","0,10","0,49","0,61","0,07","0,27","0,08","0,04","0,92",nan,"0,18","0,27"
Pellets industriels (gpi),nan,nan,"0,04","0,82","0,11","0,01",nan,nan,nan,"0,26",nan,"0,15","0,17"
Mégots et filtres à cigarettes,"0,09","0,32","0,12","0,42","0,66","0,34","0,19","0,06","0,54","0,58","0,05","1,58","0,27"
"Emballages de bonbons, de snacks","0,31","0,10","0,24","0,26","0,23","0,62","0,29","0,14","0,13","0,20","0,08","0,65","0,22"
Bâtonnets de sucette,"0,03","0,25",nan,"0,02","0,04","0,08","0,07","0,03","0,02","0,10",nan,"0,06","0,06"
Mousse de plastique pour l'isolation thermique,nan,nan,"0,04",nan,"0,10","0,07",nan,nan,"0,05","0,13",nan,"0,06","0,07"
Déchets de construction en plastique,"0,05",nan,nan,"0,03","0,05","0,02","0,02",nan,"0,01","0,05",nan,nan,"0,04"
Coton-tige,"0,14",nan,"0,06","0,09","0,15","0,23","0,15",nan,"0,01","0,20",nan,"0,30","0,19"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,07","0,04",nan,"0,03","0,16","0,31","0,14",nan,nan,"0,13","0,06","0,16","0,10"
"Fragments de polystyrène expansé: g76, g81, g82, g83","0,46","0,09","0,36","0,18","0,16","0,15","0,02","0,07","0,07","0,05",nan,"0,01","0,14"


### Les plus courant Plastock - IQAASL

In [9]:
boundaries = dict(start_date="2019-12-31", end_date="2023-01-01", feature_name="lac-leman", language="fr")

combined = rc.ReportClass(txi.copy(), boundaries=boundaries, language="fr", lang_maps=language_maps)
t = rc.a_cumulative_report(combined.w_df[combined.w_df.code.isin(most_common.index)], feature_name='project', object_column='code')
rc.translated_and_style_for_display(t, tiii.lang_maps[tiii.language], tiii.language, gradient=True)

,Iqaasl,Plastock,Cumulé
Fragments de plastique angulaires <5mm,"0,00","0,27","0,00"
Pellets industriels (gpi),"0,00","0,17","0,03"
Mégots et filtres à cigarettes,"0,47","0,27","0,35"
"Emballages de bonbons, de snacks","0,22","0,22","0,22"
Bâtonnets de sucette,"0,03","0,06","0,04"
Mousse de plastique pour l'isolation thermique,"0,10","0,07","0,09"
Déchets de construction en plastique,"0,06","0,04","0,05"
Coton-tige,"0,13","0,19","0,16"
"Couvercles en plastique bouteille: g21, g22, g23, g24","0,11","0,10","0,11"
"Fragments de polystyrène expansé: g76, g81, g82, g83","0,24","0,14","0,17"


## Inventaire

In [10]:
t = rc.translate_for_display(tiii.inventory, amap=language_maps['fr'], lan='fr')
t['objet'] = t.code.apply(lambda x: codes.loc[x, 'fr'])
t = t[[t.columns[0], t.columns[-1], *t.columns[1:-1]]]
t.set_index(['code', 'objet'], inplace=True)
t.index.name = None
t.style.set_table_styles(conf_.table_css_styles).format(**conf_.format_kwargs)

,,quantité,% du total,pcs/m,taux d'échec
code,objet,,,,
Gfrags,"Fragments de plastique: G80, G79, G78, G75",11'221,"0,41","0,32","0,97"
G27,Mégots et filtres à cigarettes,3'089,"0,11","0,12","0,79"
G30,"Emballages de bonbons, de snacks",2'080,"0,08","0,13","0,74"
G106,Fragments de plastique angulaires <5mm,1'926,"0,07","0,17","0,41"
G112,Pellets industriels (GPI),1'526,"0,06","0,10","0,36"
Gfoams,"Fragments de polystyrène expansé: G76, G81, G82, G83",1'399,"0,05","0,09","0,72"
Gcaps,"Couvercles en plastique bouteille: G21, G22, G23, G24",1'070,"0,04","0,06","0,65"
G95,Coton-tige,1'040,"0,04","0,11","0,54"
G74,Mousse de plastique pour l'isolation thermique,406,"0,01","0,05","0,38"


In [11]:
%watermark --iversions -b -r

Git repo: https://github.com/hammerdirt-analyst/plastock.git

Git branch: nov28

seaborn   : 0.12.2
numpy     : 1.24.2
matplotlib: 3.7.1
pandas    : 2.0.0

